In [8]:
# Import necessary libraries
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector
from mesa.visualization.UserParam import UserSettableParameter # Object from mesa version 1.2.1
import random


In [9]:
# Define TreeCell agent that represents a tree in the forest
class TreeCell(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.burning = False  # Indicates if the tree is burning
        self.burned_out = False  # Indicates if the tree is burned out
        self.burn_time = 0  # Time the tree has been burning
        self.extinguished = False  # Indicates if the tree has been extinguished by a firefighter

    def step(self):
        if self.burning:
            self.burn_time += 1
            # Spread fire to neighbors based on Moore neighborhood
            neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=True)
            non_burning_neighbors = [neighbor for neighbor in neighbors if isinstance(neighbor, TreeCell) and not neighbor.burning and not neighbor.burned_out and not neighbor.extinguished]
            if non_burning_neighbors:
                chosen_neighbor = random.choice(non_burning_neighbors)
                chosen_neighbor.burning = True

            # If burning for 3 time steps, tree burns out
            if self.burn_time >= 3:
                self.burning = False
                self.burned_out = True


In [10]:
# Define Firefighter agent that moves around the forest to extinguish fires
class Firefighter(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        # Get the Moore neighborhood
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=True)
        # Prioritize moves to cells with burning trees
        burning_trees = [move for move in possible_moves if any(isinstance(agent, TreeCell) and agent.burning for agent in self.model.grid.get_cell_list_contents([move]))]

        if burning_trees:
            new_position = random.choice(burning_trees)
        else:
            new_position = random.choice(possible_moves)

        # Move to the new position
        self.model.grid.move_agent(self, new_position)

        # Extinguish fire if present
        cellmates = self.model.grid.get_cell_list_contents([new_position])
        for cellmate in cellmates:
            if isinstance(cellmate, TreeCell) and cellmate.burning:
                cellmate.burning = False
                cellmate.burn_time = 0
                cellmate.extinguished = True
                print(f"Firefighter {self.unique_id} extinguished fire at {new_position}")


In [11]:
# Functions to collect data on the model's state
def get_burning_trees(model):
    return len([agent for agent in model.schedule.agents if isinstance(agent, TreeCell) and agent.burning])

def get_burned_out_trees(model):
    return len([agent for agent in model.schedule.agents if isinstance(agent, TreeCell) and agent.burned_out])

def get_extinguished_trees(model):
    return len([agent for agent in model.schedule.agents if isinstance(agent, TreeCell) and agent.extinguished])

def get_remaining_trees(model):
    return len([agent for agent in model.schedule.agents if isinstance(agent, TreeCell) and not agent.burning and not agent.burned_out and not agent.extinguished])


In [12]:
# Define the FireModel that represents the forest fire simulation
class FireModel(Model):
    def __init__(self, width, height, density=0.9, num_firefighters=5): # Default value parameter
        super().__init__()
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.num_firefighters = num_firefighters

        # Initialize DataCollector
        self.datacollector = DataCollector(
            model_reporters={
                "Burning Trees": get_burning_trees,
                "Burned Out Trees": get_burned_out_trees,
                "Extinguished Trees": get_extinguished_trees,
                "Remaining Trees": get_remaining_trees
            }
        )

        # Create tree cells
        for x in range(self.grid.width):
            for y in range(self.grid.height):
                if random.random() < density:
                    tree = TreeCell((x, y), self)
                    self.grid.place_agent(tree, (x, y))
                    self.schedule.add(tree)

        # Initialize fire at random locations
        self.init_fire()

        # Create firefighter agents
        for i in range(self.num_firefighters):
            firefighter = Firefighter(i, self)
            self.schedule.add(firefighter)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(firefighter, (x, y))

    def init_fire(self):
        initial_fire_count = 0
        while initial_fire_count < 10: # Number of initial burn tree
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            cell_contents = self.grid.get_cell_list_contents([(x, y)])
            for agent in cell_contents:
                if isinstance(agent, TreeCell) and not agent.burning:
                    agent.burning = True
                    initial_fire_count += 1

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [13]:
# Define how agents are portrayed in the visualization
def agent_portrayal(agent):
    if isinstance(agent, TreeCell):
        portrayal = {"Shape": "rect", "w": 1, "h": 1, "Filled": "true"}
        if agent.burned_out:
            portrayal["Color"] = "black"
        elif agent.burning:
            portrayal["Color"] = "red"
        elif agent.extinguished:
            portrayal["Color"] = "yellow"
        else:
            portrayal["Color"] = "green"
        portrayal["Layer"] = 0
    elif isinstance(agent, Firefighter):
        portrayal = {"Shape": "circle", "r": 0.5, "Color": "blue", "Layer": 1}
    return portrayal


In [14]:
# Define parameters that the user can modify through the web interface
model_params = {
    "width": 25,
    "height": 25,
    "density": UserSettableParameter('slider', 'Forest Density', 0.9, 0.1, 1.0, 0.1),
    "num_firefighters": UserSettableParameter('slider', 'Number of Firefighters', 5, 1, 125, 1)
}

# Create visualization elements
grid = CanvasGrid(agent_portrayal, model_params["width"], model_params["height"], 500, 500)
chart = ChartModule([{"Label": "Burning Trees", "Color": "Red"},
                     {"Label": "Burned Out Trees", "Color": "Black"},
                     {"Label": "Extinguished Trees", "Color": "Yellow"},
                     {"Label": "Remaining Trees", "Color": "Green"}],
                    data_collector_name='datacollector')

# Create the server
server = ModularServer(
    FireModel,
    [grid, chart],
    "Fire Suppression Model",
    model_params
)

# Specify the port and launch the server
server.port = 2584
server.launch()

Interface starting at http://127.0.0.1:2584


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"submit_params","param":"num_firefighters","value":8}
{"type":"submit_params","param":"num_firefighters","value":9}
{"type":"submit_params","param":"num_firefighters","value":13}
{"type":"submit_params","param":"num_firefighters","value":17}
{"type":"submit_params","param":"num_firefighters","value":20}
{"type":"submit_params","param":"num_firefighters","value":21}
{"type":"submit_params","param":"num_firefighters","value":22}
{"type":"submit_params","param":"num_firefighters","value":23}
{"type":"submit_params","param":"num_firefighters","value":24}
{"type":"submit_params","param":"num_firefighters","value":27}
{"type":"submit_params","param":"num_firefighters","value":30}
{"type":"submit_params","param":"num_firefighters","value":31}
{"type":"submit_params","param":"num_firefighters","value":32}
{"type":"submit_params","param":"num_firefighters","value":34}
{"type":"submit_params","param":"num_firefighters","value":35}
{"type":"submit_params","